In [1]:
#set openai key in .env.local
import os
import pandas as pd
import openai
import numpy as np
import tensorflow as tf
import regex as re
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding

#from openai.embeddings_utils import get_embedding

In [2]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'no_product_variants.pkl')
product_data = product_data.reset_index(drop=True)
# product_data = product_data.sample(n=10, random_state=2).reset_index(drop=True)
display(product_data)

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,40.0,schwarz,02.643.57,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0
1,551622,4054452768212,elegante Pumps Glattleder schwarz,38.0,schwarz,05.160.37,6590677,315573,w,PT,...,Pumps,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0
2,547193,4054452768427,Slipper Glattleder schwarz,42.0,schwarz,04.443.27,6590677,315576,w,SK,...,Halbschuhe,Damenschuhe,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0
3,549776,4059701687894,Sneaker low Rauleder blau,39.0,blau,06.968.46,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,37.5,schwarz,06.102.67,6590678,315574,w,SK,...,Ballerinas,Damenschuhe,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,35.0,blauw,24.764.36,6590677,315568,w,SK,...,Sandalen,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,40.5,pink,26.090.21,6590678,315590,w,PT,...,Halbschuhe,Damenschuhe,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0
3294,10782704,4251234499207,Shopper ANDIE blau,,blau,921453,363013,363017,w,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3295,10679703,4066558303617,Elegante pumps Glad leer wit,35.0,wit,21.450.60,6590677,315573,w,PT,...,Pumps,Damenschuhe,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0


In [3]:
product_data[product_data['product__id'].isin([7594042,3505964])]

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
100,3505964,4060666978859,Sneaker low Materialmix Leder/Lederimitat grau,38.0,grau,56.946.39,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,Gummi-EVA,0.0,0.0
2182,7594042,4064032158098,Sneaker low Materialmix Leder/Lederimitat Grau,38.0,Grau,86.986.40,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,Gummi-EVA,0.0,0.0


In [4]:
def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def combine_columns(row):
    return preprocess_text (
            "ID: " + str(row.product__id) + "; " +
            "product_number: " + str(row.productNumber) + "; " +
            "product_name: " + row.productName + "; " +
            # "size: " + str(row.productSizeEu) + "; " +
            "color: " + str(row.productColorName) + "; " +
            "article number: " + str(row.articleNumber) + "; " +
            "brand: " + str(row.brand__id) + "; " +
            "main category: " + str(row.mainCategory__id) + "; " +
            "gender: " + str(row.gender) + "; " +
            "line: " + str(row.line) + "; " +
            "price: " + str(row.b2cPrice) + "; " +
            "material: " + str(row.material) + "; " +
            "material PM: " + str(row.materialPm) + "; "
    )


product_data_embeddings = product_data[['product__id']].copy()
product_data_embeddings["combined"] = product_data.apply(lambda row: combine_columns(row), axis=1)
display(product_data_embeddings)

,product__id,combined
0,557559,id 557559 product_number 4058394021466 product...
1,551622,id 551622 product_number 4054452768212 product...
2,547193,id 547193 product_number 4054452768427 product...
3,549776,id 549776 product_number 4059701687894 product...
4,550726,id 550726 product_number 4054452851594 product...
...,...,...
3292,10603553,id 10603553 product_number 4066558951733 produ...
3293,10442948,id 10442948 product_number 4065171827272 produ...
3294,10782704,id 10782704 product_number 4251234499207 produ...
3295,10679703,id 10679703 product_number 4066558303617 produ...


In [5]:
vectorizer = TextVectorization(output_mode='int', output_sequence_length=10, standardize=None)

# Apply the custom preprocessing function to the 'comb
# Update the vocabulary based on the preprocessed text
vectorizer.adapt(product_data_embeddings['combined'].values)
vocab_size = len(vectorizer.get_vocabulary())

In [6]:
import os
import requests
import zipfile
import numpy as np

def download_and_extract_glove(glove_url, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    glove_zip = os.path.join(save_dir, 'glove.6B.zip')
    glove_txt = os.path.join(save_dir, 'glove.6B.100d.txt')

    if not os.path.exists(glove_txt):
        print("Downloading GloVe embeddings...")
        response = requests.get(glove_url, stream=True)
        with open(glove_zip, 'wb') as f:
            f.write(response.content)

        print("Extracting GloVe embeddings...")
        with zipfile.ZipFile(glove_zip, 'r') as zip_ref:
            zip_ref.extractall(save_dir)

        os.remove(glove_zip)

# Replace 'save_directory' with the path to the directory where you want to save the embeddings
save_directory = '.'
glove_url = 'https://nlp.stanford.edu/data/glove.6B.zip'
download_and_extract_glove(glove_url, save_directory)

In [7]:
glove_txt = os.path.join(save_directory, 'glove.6B.100d.txt')
embeddings_index = {}
with open(glove_txt, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_dim = 100  # You can choose the desired embedding dimension

for token in vectorizer.get_vocabulary():
    if token.isnumeric():  # Check if the token is a number
        # Handle out-of-vocabulary numeric tokens
        # This example assigns a random vector to each numeric token
        embeddings_index[token] = np.random.rand(embedding_dim)

# Define the Embedding layer using the pre-trained GloVe embeddings

# Proceed to build your embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
word_index = {word: index for index, word in enumerate(vectorizer.get_vocabulary())}

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        # If a specific number isn't in GloVe, assign a random vector (or handle as needed)
        embedding_matrix[i] = np.random.rand(embedding_dim)

# Create the embedding layer as before
embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False
)



In [8]:
# Convert the preprocessed text to numeric vectors
numeric_vectors = vectorizer(product_data_embeddings['combined'].values)

# Get word embeddings
word_embeddings = embedding_layer(numeric_vectors)
word_embeddings, word_embeddings.shape

(<tf.Tensor: shape=(3297, 10, 100), dtype=float32, numpy=
 array([[[-0.22458   , -0.58967   ,  0.43007   , ..., -0.63613   ,
           0.31284   , -0.82294   ],
         [ 0.6953023 ,  0.4240106 ,  0.34703028, ...,  0.7000183 ,
           0.7787662 ,  0.03887276],
         [ 0.66505736,  0.24711573,  0.8488608 , ...,  0.9680706 ,
           0.5628829 ,  0.27689692],
         ...,
         [ 0.84386545,  0.306656  ,  0.71042216, ...,  0.41516533,
           0.75955874,  0.60173255],
         [ 0.53366   ,  0.0085679 , -0.85512   , ...,  0.21118   ,
           0.22603   ,  0.18089   ],
         [-0.52516   ,  0.44118   , -0.1273    , ..., -0.88581   ,
           0.075446  ,  0.040447  ]],
 
        [[-0.22458   , -0.58967   ,  0.43007   , ..., -0.63613   ,
           0.31284   , -0.82294   ],
         [ 0.3563039 ,  0.9010375 ,  0.08775103, ...,  0.29836118,
           0.4368072 ,  0.9831244 ],
         [ 0.66505736,  0.24711573,  0.8488608 , ...,  0.9680706 ,
           0.5628829 ,  0.

In [29]:
# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
from sklearn.metrics.pairwise import cosine_similarity

def get_n_nearest_neighbors(query_embedding, neighbors_embedding, n = 5):
    input_embedding_flat = query_embedding.numpy().reshape(query_embedding.shape[0], -1)
    word_embeddings_flat = neighbors_embedding.numpy().reshape(neighbors_embedding.shape[0], -1)
    # display(input_embedding_flat.shape, word_embeddings_flat.shape)
    # Calculate the cosine similarity between the input embedding and all other word embeddings
    similarities = cosine_similarity(input_embedding_flat, word_embeddings_flat)
    # Get the indices of the most similar neighbors
    neighbor_indices = np.argsort(similarities[0])[::-1][:n]

    scores = [similarities[0][i] for i in neighbor_indices]
   
    return neighbor_indices, scores


indices = get_n_nearest_neighbors(tf.gather(word_embeddings, [240]), word_embeddings)
display(indices)

(array([ 240,  100, 2182,  200,  218]),
 [0.9999999, 0.9210142, 0.9170989, 0.90137005, 0.8880319])

In [13]:
product_data_embeddings[product_data_embeddings['product__id'] == 3696208]

,product__id,combined
240,3696208,id 3696208 product_number 4062862013129 produc...


In [34]:
product_data_embeddings.iloc[indices[0]]

,product__id,combined
240,3696208,id 3696208 product_number 4062862013129 product_name sneaker low materialmix ledertextil grau color grau article number 10071003 brand 3692257 main category 3692261 gender m line hw price 1200 material materialmix ledertextil material pm suedevelvetnylon
100,3505964,id 3505964 product_number 4060666978859 product_name sneaker low materialmix lederlederimitat grau color grau article number 5694639 brand 4997827 main category 315567 gender w line hw price 9995 material materialmix lederlederimitat material pm mesh modeldreamv
2182,7594042,id 7594042 product_number 4064032158098 product_name sneaker low materialmix lederlederimitat grau color grau article number 8698640 brand 4997827 main category 315567 gender w line fs price 1300 material materialmix lederlederimitat material pm meshloopvelourch
200,3695392,id 3695392 product_number 4062862138082 product_name sneaker low materialmix ledertextil blau color blau article number 04961001 brand 3692257 main category 3692261 gender m line hw price 9995 material materialmix ledertextil material pm suedecanvas
218,3696172,id 3696172 product_number 4062862012818 product_name sneaker low materialmix ledertextil schwarz color schwarz article number 10071001 brand 3692257 main category 3692261 gender m line hw price 1200 material materialmix ledertextil material pm suedevelvetnylon


In [14]:
pd.set_option('display.max_colwidth', None)
print(indices)
for idx in indices[0]:
    display(product_data_embeddings.iloc[idx]['combined'], '\n')

(array([ 240,  100, 2182,  200,  218]), [0.9999999, 0.9210142, 0.9170989, 0.90137005, 0.8880319])


'id 3696208 product_number 4062862013129 product_name sneaker low materialmix ledertextil grau color grau article number 10071003 brand 3692257 main category 3692261 gender m line hw price 1200 material materialmix ledertextil material pm suedevelvetnylon '

'\n'

'id 3505964 product_number 4060666978859 product_name sneaker low materialmix lederlederimitat grau color grau article number 5694639 brand 4997827 main category 315567 gender w line hw price 9995 material materialmix lederlederimitat material pm mesh modeldreamv '

'\n'

...

'\n'

'id 3695392 product_number 4062862138082 product_name sneaker low materialmix ledertextil blau color blau article number 04961001 brand 3692257 main category 3692261 gender m line hw price 9995 material materialmix ledertextil material pm suedecanvas '

'\n'

'id 3696172 product_number 4062862012818 product_name sneaker low materialmix ledertextil schwarz color schwarz article number 10071001 brand 3692257 main category 3692261 gender m line hw price 1200 material materialmix ledertextil material pm suedevelvetnylon '

'\n'

In [15]:
product_data.iloc[indices]

,product__id,product__id,product__id,product__id,product__id
240,3696208,3696208,3696208,3696208,3696208
100,3505964,3505964,3505964,3505964,3505964
2182,7594042,7594042,7594042,7594042,7594042
200,3695392,3695392,3695392,3695392,3695392
218,3696172,3696172,3696172,3696172,3696172


In [19]:


def get_best_recommendations_multiple(query_embeddings, neighbors_embeddings, n = 5):
    all_results = {}
    
    for query_embedding in query_embeddings:
        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)
        result_dict = {index: score for index, score in zip(indices, scores)}
        for key, value in result_dict.items():
            if key in all_results:
                all_results[key] = max(all_results[key], value)
            else: 
                all_results[key] = value
    return dict(sorted(all_results.items(), key=lambda item: item[1], reverse=True)[:n])


def split_embeddings(idxs, embeddings): 

    selected_entries = tf.gather(embeddings, idxs)
    
    # Create a boolean mask to mark the indices to keep
    mask = tf.ones(tf.shape(embeddings)[0], dtype=tf.bool)
    mask = tf.tensor_scatter_nd_update(mask, tf.expand_dims(idxs, axis=1), tf.zeros_like(idxs, dtype=tf.bool))
    
    # Use boolean mask to get the filtered tensor
    filtered_tensor = tf.boolean_mask(embeddings, mask)
    print(filtered_tensor.shape, embeddings.shape, selected_entries.shape)
    print(filtered_tensor)
    
    return selected_entries, filtered_tensor

bought_embeddings, search_embeddings = split_embeddings([0, 1], word_embeddings)
#this does not work that great, because the orginal index gets lost, better let some more entries return and ignore the once with 1.0 score

display(bought_embeddings.shape, search_embeddings.shape)

best_items = get_best_recommendations_multiple([bought_embeddings], search_embeddings)
            
print(best_items)
display(list(best_items.keys()), product_data.iloc[list(best_items.keys())])

(3295, 10, 100) (3297, 10, 100) (2, 10, 100)
tf.Tensor(
[[[-2.2458000e-01 -5.8967000e-01  4.3007001e-01 ... -6.3612998e-01
    3.1284001e-01 -8.2293999e-01]
  [ 3.6685815e-01  4.2948115e-01  2.5171143e-01 ...  9.2626011e-01
    7.8076041e-01  3.0607390e-01]
  [ 6.6505736e-01  2.4711573e-01  8.4886080e-01 ...  9.6807063e-01
    5.6288290e-01  2.7689692e-01]
  ...
  [ 5.3365999e-01  8.5679004e-03 -8.5512000e-01 ...  2.1118000e-01
    2.2603001e-01  1.8088999e-01]
  [-5.2516001e-01  4.4117999e-01 -1.2729999e-01 ... -8.8581002e-01
    7.5446002e-02  4.0447000e-02]
  [ 5.3365999e-01  8.5679004e-03 -8.5512000e-01 ...  2.1118000e-01
    2.2603001e-01  1.8088999e-01]]

 [[-2.2458000e-01 -5.8967000e-01  4.3007001e-01 ... -6.3612998e-01
    3.1284001e-01 -8.2293999e-01]
  [ 9.1686028e-01  3.6507404e-01  5.9892732e-01 ...  9.7123677e-01
    5.8610481e-01  3.6431726e-02]
  [ 6.6505736e-01  2.4711573e-01  8.4886080e-01 ...  9.6807063e-01
    5.6288290e-01  2.7689692e-01]
  ...
  [ 8.2696813e-01  7.

TensorShape([2, 10, 100])

TensorShape([3295, 10, 100])

(2, 1000)

(3295, 1000)

{31: 0.92777663, 3037: 0.92112637, 1235: 0.9171039, 6: 0.9136152, 1809: 0.90605056}


[31, 3037, 1235, 6, 1809]

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
31,556962,4054452768830,Eleganter Pump Glattleder schwarz,43.0,schwarz,05.180.37,6590677,315573,w,SK,...,Pumps,Damenschuhe,5 cm - 8 cm,43.0,9.0,16.0,0.0,EVA,0.0,0.0
3037,10560404,4065171981417,Sneaker low Textil rosa,39.0,rosa,26.916.35,4997827,315567,w,VN,...,Sneaker,Damenschuhe,5 cm - 8 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0
1235,4913510,4062862655992,Sneaker low Textil Schwarz,36.0,Schwarz,63.482.17,6590677,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,36.0,3.5,5.0,0.0,EVA,0.0,0.0
6,407469,4054452837581,elegante Ballerinas Glattleder schwarz,38.5,schwarz,04.111.27,6590677,315574,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,38.5,5.5,9.0,0.0,TPU,0.0,0.0
1809,7870543,4064032931493,Sportliche Stiefelette Rauleder grün,39.0,grün,52.711.33,6590678,5014851,w,PT,...,Stiefeletten,Damenschuhe,3 cm - 5 cm,39.0,6.0,10.0,14.0,Gummi,0.0,1.0


In [ ]:
best_items = get_best_recommendations_multiple([bought_embeddings], word_embeddings, 7)

print(best_items)

display(product_data.iloc[list(best_items.keys())])

<tf.Tensor: shape=(3297, 10, 100), dtype=float32, numpy=
array([[[-0.22458   , -0.58967   ,  0.43007   , ..., -0.63613   ,
          0.31284   , -0.82294   ],
        [ 0.6953023 ,  0.4240106 ,  0.34703028, ...,  0.7000183 ,
          0.7787662 ,  0.03887276],
        [ 0.66505736,  0.24711573,  0.8488608 , ...,  0.9680706 ,
          0.5628829 ,  0.27689692],
        ...,
        [ 0.84386545,  0.306656  ,  0.71042216, ...,  0.41516533,
          0.75955874,  0.60173255],
        [ 0.53366   ,  0.0085679 , -0.85512   , ...,  0.21118   ,
          0.22603   ,  0.18089   ],
        [-0.52516   ,  0.44118   , -0.1273    , ..., -0.88581   ,
          0.075446  ,  0.040447  ]],

       [[-0.22458   , -0.58967   ,  0.43007   , ..., -0.63613   ,
          0.31284   , -0.82294   ],
        [ 0.3563039 ,  0.9010375 ,  0.08775103, ...,  0.29836118,
          0.4368072 ,  0.9831244 ],
        [ 0.66505736,  0.24711573,  0.8488608 , ...,  0.9680706 ,
          0.5628829 ,  0.27689692],
        ..

In [31]:


def evaluate_coverage_tensor(embeddings_tensor, n=5):
    recommended_items = set()
    scores_list = []
    total_items = embeddings_tensor.shape[0]

    for i in range(total_items):
        # Exclude the current query embedding from the neighbors
        neighbors_embeddings = tf.concat([embeddings_tensor[:i], embeddings_tensor[i+1:]], axis=0)

        query_embedding = embeddings_tensor[i:i+1]
        indices, scores = get_n_nearest_neighbors(query_embedding, neighbors_embeddings, n)

        # Convert indices and scores to numpy arrays if they are TensorFlow tensors
        if isinstance(indices, tf.Tensor):
            indices = indices.numpy().flatten()
        if isinstance(scores, tf.Tensor):
            scores = scores.numpy()

        recommended_items.update(indices)
        scores_list.extend(scores)
        if i % 100 == 0:
            print(i)

    coverage_score = len(recommended_items) / (total_items - 1)

    return coverage_score, scores_list

# Example usage
coverage_score, confidence_scores = evaluate_coverage_tensor(word_embeddings)
print(f"Coverage Score: {coverage_score}")
print(f"Confidence Score: {np.mean(confidence_scores)}; Min Score: {np.min(confidence_scores)}; Max Score: {np.max(confidence_scores)}")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
Coverage Score: 0.977245145631068
Confidence Scores: [0.9277761, 0.9211267, 0.9171042, 0.9136156, 0.9060511, 0.8951252, 0.8767374, 0.87388563, 0.87228084, 0.87165475, 0.94545054, 0.9452236, 0.93010616, 0.92209166, 0.92030585, 0.9407307, 0.94060075, 0.9399033, 0.92270535, 0.9225793, 0.837049, 0.8346778, 0.83275914, 0.8140385, 0.81128526, 0.92071205, 0.9189772, 0.91067016, 0.9086372, 0.89911723, 0.90764666, 0.8965565, 0.88979614, 0.8767374, 0.8605714, 0.9068092, 0.9047199, 0.88979614, 0.8752172, 0.85970783, 0.9264917, 0.92310166, 0.92226696, 0.9136156, 0.90320987, 0.90764666, 0.8993897, 0.8640983, 0.8550545, 0.8531593, 0.9047199, 0.8993897, 0.8750641, 0.8570533, 0.8423836, 0.9242635, 0.9222067, 0.91912663, 0.91793394, 0.8995912, 0.9456563, 0.9445916, 0.9444298, 0.93178654, 0.92544425, 0.9068092, 0.8965565, 0.8750641, 0.8640983, 0.8639204

In [33]:
print(f"Confidence Score: {np.mean(confidence_scores)}; Min Score: {np.min(confidence_scores)}; Max Score: {np.max(confidence_scores)}")
print(f"Coverage Score: {coverage_score}")

Confidence Score: 0.9099971652030945; Min Score: 0.6741653680801392; Max Score: 0.9652183055877686
Coverage Score: 0.977245145631068
